In [ ]:
# How to Record
# https://people.csail.mit.edu/hubert/pyaudio/

# Fixed by enabling VSCode's mic :) lmao
# https://stackoverflow.com/questions/61277449/mac-os-mojave-microphone-permission-for-visual-studio-code

import wave
import sys

import pyaudio

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1 if sys.platform == 'darwin' else 2
RATE = 44100
RECORD_SECONDS = 5

with wave.open('output.wav', 'wb') as wf:
    p = pyaudio.PyAudio()
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)

    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True)

    print('Recording...')
    for _ in range(0, RATE // CHUNK * RECORD_SECONDS):
        wf.writeframes(stream.read(CHUNK))
    print('Done')

    stream.close()
    p.terminate()

In [ ]:
import openai

# Set your OpenAI API key
# openai.api_key

# Open the audio file you want to transcribe
file_path = ""
audio_file = open(file_path, "rb")

# Use the openai.Audio class to create a transcription
response = openai.Audio.transcribe(
    model="whisper-1", file=audio_file, response_format="text"
)

# Print the transcription result
print(response)

Weird - above only works if you convert it to .mp3 first

Used terminal:
`ffmpeg -i "/Users/cvk/Downloads/[CODE] Local Projects/24SUMR_Recommender-Algo/Q&A_WhisperOnWeb/padayonko.m4a" "/Users/cvk/Downloads/[CODE] Local Projects/24SUMR_Recommender-Algo/Q&A_WhisperOnWeb/padayonko.mp3"`



In [27]:
import simpleaudio as sa
import openai
import pyaudio
import wave
import tempfile
import os

def transcribe_audio(runPlayback=False):
    """
    Records audio directly from the microphone and transcribes it to text using OpenAI's API.

    Returns:
        str: The transcription of the recorded audio.
    """
    # Create a temporary file to store the recorded audio
    temp_file = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    temp_file_name = temp_file.name

    sample_rate = 16000
    bits_per_sample = 16
    chunk_size = 1024
    audio_format = pyaudio.paInt16
    channels = 1

    def callback(in_data, frame_count, time_info, status):
        wav_file.writeframes(in_data)
        return None, pyaudio.paContinue

    # Open the wave file for writing
    with wave.open(temp_file_name, 'wb') as wav_file:
        wav_file.setnchannels(channels)
        wav_file.setsampwidth(bits_per_sample // 8)
        wav_file.setframerate(sample_rate)

        # Initialize PyAudio and start recording audio
        audio = pyaudio.PyAudio()
        stream = audio.open(format=audio_format,
                            channels=channels,
                            rate=sample_rate,
                            input=True,
                            frames_per_buffer=chunk_size,
                            stream_callback=callback)

        input("Press Enter to stop recording...")

        # Stop and close the audio stream
        stream.stop_stream()
        stream.close()
        audio.terminate()

    # Play the recorded audio
    if runPlayback:
        playback = sa.WaveObject.from_wave_file(temp_file.name)
        play_obj = playback.play()
        play_obj.wait_done()  # Wait until playback is finished

    # Transcribe the audio to text using OpenAI's API
    with open(temp_file_name, 'rb') as audio_file:
        response = openai.Audio.transcribe(
            file=audio_file,
            model="whisper-1",
            prompt="Carl talks to Hadavand"
        )
        transcription = response['text'].strip()

    # Delete the temporary audio file
    os.remove(temp_file_name)

    return transcription

In [28]:
transcription = transcribe_audio(runPlayback=True)
print("Transcription:", transcription)

Transcription: Hi, my name is Karl and I'm going to be talking to Prof. Hadavand. Do you hear me, Sarah?
